In [1]:
import sys
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install ase torch_geometric
    import torch
    from torch_geometric.data import Data, DataLoader

    dataset = "/content/drive/My Drive/Dataset"
else:
    dataset = "Dataset"

In [ ]:
!pip install pymatgen torch-geometric ase

In [ ]:
import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from ase.data import atomic_masses, covalent_radii
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from pymatgen.core import Structure

In [2]:
import pandas as pd
import numpy as np

In [3]:
data_df = pd.read_pickle(f"{dataset}/absorption_mp_data.pkl")
data_df

,id,formula,structure,energies,absorption_coefficient,imag_dielectric,real_dielectric,energy_max,bandgap
0,mp-546266,DyBi2IO4,"(Atom('Dy', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0277, 0.0554, 0.0831, 0.1109, 0.1386, ...","[0.0, 0.8362304735329499, 3.309178082718349, 7...","[0.0, 0.0015666666666666665, 0.0031, 0.0046666...","[6.9163, 6.916533333333334, 6.9172, 6.9182, 6....",55.4328,1.3818000000000001
1,mp-9583,K2ZnF4,"(Atom('K', [np.float64(1.2531985199117757), np...","[0.0, 0.0399, 0.0798, 0.1197, 0.1596, 0.1995, ...","[0.0, 0.1362578822994739, 0.4541860898852432, ...","[0.0, 0.0001, 0.00016666666666666666, 0.0002, ...","[2.2021333333333333, 2.202166666666667, 2.2022...",79.7804,4.3866
2,mp-22988,CsGeCl3,"(Atom('Cs', [np.float64(7.78333021211009), np....","[0.0, 0.029, 0.058, 0.087, 0.116, 0.1449, 0.17...","[0.0, 0.515935972729374, 2.0145311056971553, 4...","[0.0, 0.0007, 0.0013666666666666669, 0.0020333...","[3.975733333333333, 3.9758333333333336, 3.9761...",57.9759,2.178
3,mp-861502,AcFeO3,"(Atom('Ac', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0587, 0.1174, 0.176, 0.2347, 0.2934, 0...","[0.0, 11295.053197761259, 44864.495977063896, ...","[0.0, 30.1914, 49.9196, 37.98413333333333, 21....","[67.54320000000001, 59.6215, 29.6067, 1.281, -...",117.3625,0.0
4,mp-1025029,PrHSe,"(Atom('Pr', [np.float64(-2.0265362), np.float6...","[0.0, 0.0365, 0.073, 0.1095, 0.1459, 0.1824, 0...","[0.0, 1.6849892209331498, 6.697792903781345, 1...","[0.0, 0.0026999999999999997, 0.005366666666666...","[8.784633333333334, 8.7851, 8.786566666666667,...",72.9718,1.9455999999999998
...,...,...,...,...,...,...,...,...,...
935,mp-733,GeO2,"(Atom('Ge', [np.float64(1.3730217033234977), n...","[0.0, 0.0392, 0.0783, 0.1175, 0.1567, 0.1958, ...","[0.0, 0.32852818576627957, 1.3124033386526137,...","[0.0, 0.0003, 0.0006, 0.0009000000000000001, 0...","[3.2907333333333333, 3.290766666666667, 3.2909...",78.3376,3.2348
936,mp-1065514,YTlTe2,"(Atom('Y', [np.float64(12.051129954999999), np...","[0.0, 0.0391, 0.0781, 0.1172, 0.1562, 0.1953, ...","[0.0, 3.930355669823682, 15.739175717204319, 3...","[0.0, 0.006433333333333333, 0.0129, 0.0194, 0....","[10.518099999999999, 10.519366666666668, 10.52...",78.1154,0.6338999999999997
937,mp-7439,K3Cu3P2,"(Atom('K', [np.float64(6.790361937514462), np....","[0.0, 0.0587, 0.1175, 0.1762, 0.2349, 0.2937, ...","[0.0, 10.698002422282322, 43.080361666969445, ...","[0.0, 0.010366666666666666, 0.0208666666666666...","[8.3065, 8.309533333333334, 8.318600000000002,...",117.4732,1.3053
938,mp-13276,SrLiP,"(Atom('Sr', [np.float64(0.0), np.float64(0.0),...","[0.0, 0.0401, 0.0802, 0.1203, 0.1604, 0.2005, ...","[0.0, 3.9082984808102155, 15.675868999546559, ...","[0.0, 0.0057, 0.011433333333333332, 0.01723333...","[8.782833333333333, 8.783966666666666, 8.78736...",80.1856,1.3567000000000005


## Reading the dataset per a structure

In [4]:
# The structure column is already an atom
a_structure = data_df["structure"][1]
print(a_structure)

Atoms(symbols='K2ZnF4', pbc=True, cell=[[3.87855986, 0.0, -1.1469436], [-0.33916743, 3.86370258, -1.1469436], [-0.00357393, -0.00390187, 7.11934679]])


## Adding unit_cell_volume row in the whole dataset{Column}

In [5]:
def get_mean(row):
    row["unit_cell_volume"] = row["structure"].get_volume()
    return row

data_df = data_df.apply(lambda row: get_mean(row), axis=1)

## What the whole structure gives us

In [6]:
print(a_structure.get_all_distances())
#print(a_structure.get_cell())
print(a_structure.get_chemical_formula())
#print(a_structure.get_atomic_numbers())
print(a_structure.get_chemical_symbols())
#print(a_structure.get_distance(a0=0, a1=1))
#print(a_structure. get_distances(0,2))
#print(a_structure.get_global_number_of_atoms())
#print(a_structure.get_initial_charges())
#print(a_structure.get_kinetic_energy())
#print(a_structure.get_pbc())
print(a_structure.get_positions())
print(a_structure.get_volume())
#print(a_structure.get_masses()) # Get atomic mass of each atom in the structure
#print(a_structure.get_momenta()) # Get the momenta of each atom in the structure
#print(a_structure.get_moments_of_inertia()) # Get the moments of inertia in the structure
#print(a_structure.get_angular_momentum())
#print(a_structure.get_initial_magnetic_moments())
#print(a_structure.get_atomic_numbers())
#print(a_structure.get_tags())
#print(a_structure.get_angle(0,1,2))

[[0.         3.95563414 4.62617937 5.04888136 5.04888143 2.86302
  2.60031286]
 [3.95563414 0.         3.4299493  2.77035814 2.77035784 2.60031286
  2.86302   ]
 [4.62617937 3.4299493  0.         2.02229487 2.02229522 5.3266708
  2.02586651]
 [5.04888136 2.77035814 2.02229487 0.         2.85995722 4.92785317
  2.86248483]
 [5.04888143 2.77035784 2.02229522 2.85995722 0.         4.92785296
  2.86248502]
 [2.86302    2.60031286 5.3266708  4.92785317 4.92785296 0.
  3.77756265]
 [2.60031286 2.86302    2.02586651 2.86248483 2.86248502 3.77756265
  0.        ]]
F4K2Zn
['K', 'K', 'Zn', 'F', 'F', 'F', 'F']
[[ 1.25319852  1.36802722  4.23786864]
 [ 2.28261998  2.49177349  0.58759095]
 [ 0.          0.          0.        ]
 [ 1.93927993  0.         -0.5734718 ]
 [-0.16958371  1.93185129 -0.5734718 ]
 [ 2.98702846  3.26072587  2.96963795]
 [ 0.54879004  0.59907484  1.85582164]]
106.65298266178146


## Atoms

In [7]:
import ase
from ase.data import atomic_masses, covalent_radii
all_nodes = []
for i in a_structure:
    A = i.number
    mass = atomic_masses[A]
    radius = covalent_radii[A]
    the_nodes = [A, mass, radius]
    all_nodes.append(the_nodes)

print(all_nodes)

[[np.int64(19), np.float64(39.0983), np.float64(2.03)], [np.int64(19), np.float64(39.0983), np.float64(2.03)], [np.int64(30), np.float64(65.38), np.float64(1.22)], [np.int64(9), np.float64(18.998403163), np.float64(0.57)], [np.int64(9), np.float64(18.998403163), np.float64(0.57)], [np.int64(9), np.float64(18.998403163), np.float64(0.57)], [np.int64(9), np.float64(18.998403163), np.float64(0.57)]]


## Interatomic connection

In [8]:
num_atoms = a_structure.get_global_number_of_atoms()
edges = []
from_node = []
to_node = []
for i in range(num_atoms):
    for j in range(num_atoms):
        if i==j:
            pass
        else:
            from_node.append(i)
            to_node.append(j)

edges = [from_node, to_node]
print(edges)

[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], [1, 2, 3, 4, 5, 6, 0, 2, 3, 4, 5, 6, 0, 1, 3, 4, 5, 6, 0, 1, 2, 4, 5, 6, 0, 1, 2, 3, 5, 6, 0, 1, 2, 3, 4, 6, 0, 1, 2, 3, 4, 5]]


## Global Attribute

In [9]:
an_index = 0
global_attr = [data_df["unit_cell_volume"][an_index]]
print(global_attr)

[np.float64(151.13454136559216)]


## Target Attribute

In [10]:
targg = [data_df["absorption_coefficient"]]
print(targg)

[0      [0.0, 0.8362304735329499, 3.309178082718349, 7...
1      [0.0, 0.1362578822994739, 0.4541860898852432, ...
2      [0.0, 0.515935972729374, 2.0145311056971553, 4...
3      [0.0, 11295.053197761259, 44864.495977063896, ...
4      [0.0, 1.6849892209331498, 6.697792903781345, 1...
                             ...                        
935    [0.0, 0.32852818576627957, 1.3124033386526137,...
936    [0.0, 3.930355669823682, 15.739175717204319, 3...
937    [0.0, 10.698002422282322, 43.080361666969445, ...
938    [0.0, 3.9082984808102155, 15.675868999546559, ...
939    [0.0, 1.8460086114985848, 7.422592382932063, 1...
Name: absorption_coefficient, Length: 940, dtype: object]


## Now as a function that will iterate through the whole dataset

In [ ]:
def get_graphical(row):
    # Get nodes of structure
    all_nodes = []
    for i in row["structure"]:
        Z = i.number
        # Atomic mass (from ASE database)
        mass = atomic_masses[Z]
        
        # Covalent radius (from ASE database)
        radius = covalent_radii[Z]
        
        the_nodes = [Z, mass, radius]
        all_nodes.append(the_nodes)

    # Get edges        
    num_atoms = row["structure"].get_global_number_of_atoms()

    edges = []

    from_node = []
    to_node = []

    for i in range(num_atoms):
        for j in range(num_atoms):
            if i==j:
                pass
            else:
                from_node.append(i)
                to_node.append(j)

    edges=[from_node, to_node]

    # Get global attributes
    global_attr = [row["unit_cell_volume"]]

    # Get target attribute
    targg = [row["absorption_coefficient"]]
    sample = [all_nodes, edges, global_attr, targg]
    return sample


data_df = data_df.apply(lambda row: get_graphical(row), axis=1)
print(data_df[0])

[[[np.int64(66), np.float64(162.5), np.float64(1.92)], [np.int64(83), np.float64(208.9804), np.float64(1.48)], [np.int64(83), np.float64(208.9804), np.float64(1.48)], [np.int64(53), np.float64(126.90447), np.float64(1.39)], [np.int64(8), np.float64(15.999), np.float64(0.66)], [np.int64(8), np.float64(15.999), np.float64(0.66)], [np.int64(8), np.float64(15.999), np.float64(0.66)], [np.int64(8), np.float64(15.999), np.float64(0.66)]], [[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7], [1, 2, 3, 4, 5, 6, 7, 0, 2, 3, 4, 5, 6, 7, 0, 1, 3, 4, 5, 6, 7, 0, 1, 2, 4, 5, 6, 7, 0, 1, 2, 3, 5, 6, 7, 0, 1, 2, 3, 4, 6, 7, 0, 1, 2, 3, 4, 5, 7, 0, 1, 2, 3, 4, 5, 6]], [151.13454136559216], [array([0.00000000e+00, 8.36230474e-01, 3.30917808e+00, ...,
       1.81631330e+03, 1.81701042e+03, 1.81774592e+03], shape=(2001,))]]


### Creating Graph representation using PyG

In [ ]:
#Load dataset
data_df = pd.read_pickle(f"{dataset}/absorption_mp_data.pkl")  # adjust path as needed

# Add global attribute: unit cell volume
def add_volume(row):
    row["unit_cell_volume"] = row["structure"].get_volume()
    return row

data_df = data_df.apply(add_volume, axis=1)

#Convert each row to PyG graph
def row_to_graph(row):
    structure = row["structure"]
    num_atoms = structure.get_global_number_of_atoms()

    # Node features: [atomic number, mass, covalent radius]
    nodes = []
    for atom in structure:
        Z = atom.number
        nodes.append([Z, atomic_masses[Z], covalent_radii[Z]])

    x = torch.tensor(nodes, dtype=torch.float)

    # Edge indices: fully connected (excluding self-loops)
    from_node, to_node = [], []
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i != j:
                from_node.append(i)
                to_node.append(j)
    edge_index = torch.tensor([from_node, to_node], dtype=torch.long)

    # Global attribute and target
    global_attr = torch.tensor([row["unit_cell_volume"]], dtype=torch.float)
    target = torch.tensor([row["absorption_coefficient"]], dtype=torch.float)

    # Create graph object
    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=None,
        y=target,
        u=global_attr,
    )

    # Add extra metadata
    data.id = row["id"]
    data.formula = row["formula"]
    data.structure = row["structure"]
    data.energies = row["energies"]

    return data

# Convert entire DataFrame to graph list
graph_list = data_df.apply(row_to_graph, axis=1).tolist()
print(graph_list)

### Existing Data Loading and Processing

In [ ]:
#Load dataset
if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = "/content/drive/My Drive/Dataset"
else:
    dataset = "Dataset"

data_df = pd.read_pickle(f"{dataset}/absorption_mp_data.pkl")

# Add global attribute: unit cell volume
def add_volume(row):
    row["unit_cell_volume"] = row["structure"].get_volume()
    return row

data_df = data_df.apply(add_volume, axis=1)

# A function to create edges
def create_graph_from_structure(row):
    structure = row['structure']
    positions = structure.get_positions()
    atomic_numbers = structure.get_atomic_numbers()
    num_atoms = structure.get_global_number_of_atoms()

    nodes = []
    for Z in atomic_numbers:
        mass = atomic_masses[Z]
        radius = covalent_radii[Z]
        nodes.append([Z, mass, radius])

    x = torch.tensor(nodes, dtype=torch.float)

    from_node, to_node = [], []
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i != j:
                 from_node.append(i)
                 to_node.append(j)

    edge_index = torch.tensor([from_node, to_node], dtype=torch.long)

    global_attr = torch.tensor([row["unit_cell_volume"]], dtype=torch.float)
    target = torch.tensor([row["absorption_coefficient"]], dtype=torch.float)

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=None,
        y=target,
        u=global_attr,
    )

    return data

# Create the dataset instance
class CrystalGraphDataset(Dataset):
    def __init__(self, dataframe):
        super().__init__()
        self.dataframe = dataframe
        self.data_list = [create_graph_from_structure(row) for index, row in dataframe.iterrows()]

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

dataset = CrystalGraphDataset(data_df)

### Data splitting

In [ ]:
# Data Splitting
full_data_list = dataset.data_list

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_data, remaining_data = train_test_split(
    full_data_list,
    train_size=train_ratio,
    random_state=42
)

val_data, test_data = train_test_split(
    remaining_data,
    test_size=test_ratio / (val_ratio + test_ratio),
    random_state=42
)

train_loader = DataLoader(train_data, batch_size=32, shuffle=False)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

print(f"Number of training graphs: {len(train_data)}")
print(f"Number of validation graphs: {len(val_data)}")
print(f"Number of test graphs: {len(test_data)}")

### GCNConnv model

In [ ]:


# Model Definition
class GCN(torch.nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return x

# Training Loop with Validation and Loss Recording
model = GCN(in_channels=3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Lists to store loss values for plotting
train_losses = []
val_losses = []

print("\nStarting training...")
model.train()

num_epochs = 40 # Define number of epochs

for epoch in range(num_epochs):
    total_train_loss = 0
    # Training step
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Calculate average training loss for the epoch
    avg_train_loss = total_train_loss / len(train_loader.dataset)
    train_losses.append(avg_train_loss) # Store training loss

    # Validation step
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = criterion(out, batch.y)
            total_val_loss += loss.item()

    # Calculate average validation loss for the epoch
    avg_val_loss = total_val_loss / len(val_loader.dataset)
    val_losses.append(avg_val_loss) # Store validation loss

    print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    model.train() # Set the model back to training mode

# Test Evaluation
print("\nTesting...")
model.eval()
total_test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.y)
        total_test_loss += loss.item()

avg_test_loss = total_test_loss / len(test_loader.dataset)
print(f'Test Loss: {avg_test_loss:.4f}')

### Lose visualization

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss per Epoch')
plt.legend()
plt.grid(True)
plt.show()